# Bose-Hubbard Model
*****
The Bose-Hubbard Hamiltonian is given by: $$\hat{H}=-J\sum_{\langle i,j\rangle}\left(\hat{b}_{i}^\dagger\hat{b}_{j}+\hat{b}_{i}\hat{b}_{j}^\dagger\right)+\frac{U}{2}\sum_{i} \hat{n}_i\left(\hat{n}_i-\hat{I}\right)+V\sum_{\langle i,j\rangle}\hat{n}_i\hat{n}_j-\mu\sum_{i}\hat{n}_i \$$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

Suppose we have two wells and one total particle in the system. Try computing the Hamiltonian for this system with $V = 0$.

In [3]:
basis1 = basis(2,0)
basis2 = basis(2,1)
def creation(state,i):
    